<a href="https://colab.research.google.com/github/kyjhub/ML/blob/main/prediction_of_diabetes_patient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
df=pd.read_csv('diabetes_binary_5050split_health_indicators_BRFSS2015.csv')
df=df.astype('int')
df.describe()
df.shape

In [ ]:
y_label=df['Diabetes_binary']
X_features=df.drop(['Diabetes_binary', 'AnyHealthcare'], axis=1)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

fig = plt.figure(figsize=(8,5))
data = [df['BMI'], df['GenHlth'], df['MentHlth'], 
        df['PhysHlth'],df['Age'], df['Education'], df['Income']]

plt.boxplot(data, labels=['BMI', 'GenHlth', 'MentHlth', 'PhysHlth', 'Age',
                          'Education', 'Income'])
plt.show()
fig.savefig('boxplot.png', dpi=200)

In [ ]:
#BMI 이상치 제거
dset=df['BMI']
q_25=np.percentile(dset.values, 25)
q_75=np.percentile(dset.values, 75)
iqr=(q_75-q_25)*1.5
smallest=q_25-iqr
largest=q_75+iqr
outlier_index=dset[(dset<smallest) | (dset>largest)].index
print('이상치 인덱스 : ', outlier_index)
X_features.drop(outlier_index, axis=0, inplace=True)
y_label.drop(outlier_index, inplace=True)

In [ ]:
print("이상치 삭제 전",df.shape)
#outlier_index=df[df['BMI']>90].index
#df=df.drop(outlier_index, axis=0)
print("이상치 삭제 후", X_features.shape)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings('ignore')
scaler=StandardScaler()
str=['BMI','Education','Age','Income','GenHlth','MentHlth','PhysHlth']
fset=X_features[str]
X_features[str]=scaler.fit_transform(fset)
#X_features=StandardScaler().fit_transform(X_features)
#scaler = MinMaxScaler(feature_range=(0, 1))
#X_features['BMI'] = scaler.fit_transform(pd.DataFrame(X_features['BMI']))
#X_features['education'] = scaler.fit_transform(pd.DataFrame(X_features['education']))
#X_features['age'] = scaler.fit_transform(pd.DataFrame(X_features['age']))
#X_features['income'] = scaler.fit_transform(pd.DataFrame(X_features['income']))
#X_features['GenHlth'] = scaler.fit_transform(pd.DataFrame(X_features['GenHlth']))
#X_features['MentHlth'] = scaler.fit_transform(pd.DataFrame(X_features['MentHlth']))
#X_features['PhysHlth'] = scaler.fit_transform(pd.DataFrame(X_features['PhysHlth']))
X_train, X_test, y_train, y_test=train_test_split(X_features, y_label, test_size=0.2, random_state=156)

In [ ]:
X_features.head()

In [ ]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE(random_state=33)
X_train, y_train = oversample.fit_resample(X_train, y_train)
print(X_train.shape)

In [ ]:
print(df.shape)
print(df['Diabetes_binary'].value_counts())

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier

xgb_clf=XGBClassifier(n_estimators=100)
params={'booster':['gbtree'], 'n_estimators':[100], 'max_depth':[3,5,7], 
        'min_child_weight':[1, 3, 5], 'colsample_bytree':[0.5, 0.7, 0.9, 1], 
        'objective':['binary:logistic'], 'reg_alpha':[0.3, 0.5, 0.7], 'gamma':[0, 1, 2]}
#kFold=StratifiedKFold(n_splits=3, shuffle=True, random_state=50)
gridcv=GridSearchCV(xgb_clf, param_grid=params, cv=5, scoring="recall")
gridcv.fit(X_train, y_train, early_stopping_rounds=100, eval_metric="auc",
           eval_set=[(X_train, y_train),(X_test, y_test)])
print('gridsearchCV최적 파라미터:', gridcv.best_params_) 
xgb_roc_score=roc_auc_score(y_test, gridcv.predict_proba(X_test)[:,1], average='macro')
print('ROC AUC: {0:.4f}'.format(xgb_roc_score))

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve
import matplotlib.pyplot as plt
#163, 164페이지 eval_threshold코드 나와있음 그것으로 수정
def eval_set_tpp(y_test, pred=None, pred_prob=None):
    confusion=confusion_matrix(y_test, pred)
    accuracy=accuracy_score(y_test, pred)
    precision=precision_score(y_test, pred)
    recall=recall_score(y_test, pred)
    f1=f1_score(y_test, pred)
    roc_auc=roc_auc_score(y_test, pred_prob)
    print('오차행렬')
    print(confusion)
    print('정확도: {0:.4f}, 재현율: {2:.4f},\
        F1: {3:4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [ ]:
xgb_clf=XGBClassifier(booster='gbtree', n_estimators=500, random_state=50, 
                      learning_rate=0.01, max_depth=5, min_child_weight=5,
                      colsample_bytree=1, reg_alpha=0.7, objective='binary:logistic',
                      gamma=1, cv=10)
xgb_clf.fit(X_train, y_train, early_stopping_rounds=200, eval_metric="auc", eval_set=[(X_train, y_train), (X_test, y_test)])
xgb_roc_score=roc_auc_score(y_test, xgb_clf.predict_proba(X_test)[:,1], average='macro')
print('ROC AUC: {0:.4f}'.format(xgb_roc_score))

In [ ]:
xgb_pred=xgb_clf.predict(X_test)
xgb_pred_proba=xgb_clf.predict_proba(X_test)[:,1]

In [ ]:
#XGBoost Wrapper Classification 사용
from xgboost import XGBClassifier

#xgb_wrapper=XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=3)
#xgb_wrapper.fit(X_train, y_train)
#w_preds=xgb_wrapper.predict(X_test)
#w_pred_proba=xgb_wrapper.predict_proba(X_test)[:, 1]
#eval_set_tpp(y_test, w_preds, w_pred_proba)

In [ ]:
thresholds=[0.25,0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65]
for i in thresholds:
    pred_sp=[1 if x> i else 0 for x in xgb_pred_proba]
    eval_set_tpp(y_test, pred_sp, xgb_pred_proba)
    print("\n")

In [ ]:
def roc_curve_plot(y_test, pred_probs):
    fig = plt.figure(figsize=(8,8))
    fpr, tpr, threshold = roc_curve(y_test, pred_probs)
    plt.plot(fpr, tpr, label='ROC')
    plt.plot([0,1], [0,1], 'k--', label='Random')
    start, end=plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1), 2))
    plt.xlim(0,1); plt.ylim(0,1)
    plt.xlabel('FPR(1-Sensitivity)');
    plt.ylabel('TPR(Recall)')
    plt.legend()
    fig.savefig('ROC_AUC_curve.png', dpi=fig.dpi)

In [ ]:
import matplotlib.pyplot as plt
roc_curve_plot(y_test, xgb_pred_proba)

In [ ]:
def precision_recall_curve_plot(y_test=None, pred_proba_c1=None):
        # threshold ndarray와 이 threshold에 따른 정밀도, 재현율 ndarray 추출
        precisions, recalls, thresholds = precision_recall_curve( y_test, pred_proba_c1)
        # X축을 threshold값으로, Y축은 정밀도, 재현율 값으로 각각 Plot 수행. 정밀도는 점선으로 표시
        fig=plt.figure(figsize=(8,6))
        threshold_boundary = thresholds.shape[0]
        plt.plot(thresholds, precisions[0:threshold_boundary], linestyle='--', label='precision')
        plt.plot(thresholds, recalls[0:threshold_boundary],label='recall')
        # threshold 값 X 축의 Scale을 0.1 단위로 변경\n",
        start, end = plt.xlim()
        plt.xticks(np.round(np.arange(start, end, 0.1),2))
        # x축, y축 label과 legend, 그리고 grid 설정\n",
        plt.xlabel('Threshold value'); plt.ylabel('Precision and Recall value')
        plt.legend(); plt.grid()
        plt.show()
        fig.savefig('Recall_precision_value.png', dpi=fig.dpi)

In [ ]:
precision_recall_curve_plot(y_test, xgb_pred_proba)